<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph_and_Gradio(1)_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradio を用いた Web 上の AI エージェントの実装

本ノートブックでは、LangGraph を用いたツール実行（Function Calling）可能なチャットボットバックエンドの構築と、Gradio を用いた Web UI の実装プロセスを解説します。

主な学習項目は以下の通りです。
1.  **ツール定義**: Python 関数を LLM が利用可能な形式で定義する方法。
2.  **ステート管理**: LangGraph を使用し、LLM の推論結果に基づいて条件分岐（ツール実行の可否判断）を行うフローの構築。
3.  **Web UI 実装**: バックエンドのロジックを Gradio インターフェースに統合し、ブラウザ上で対話可能なアプリケーションとして公開する方法。

LLM バックエンドには OpenRouter 経由で DeepSeek モデルを使用しますが、LangChain の抽象化により他の OpenAI 互換モデルでも同様の構造で動作します。

In [ ]:
# ライブラリのインストール
!pip install -qU langgraph langchain-openai langchain-core gradio termcolor

# 環境設定

OpenRouter API キーを環境変数に設定し、モデルを初期化します。

In [ ]:
import os
from google.colab import userdata
from termcolor import colored

# APIキーの読み込み
try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    print(colored("OPENROUTER_API_KEY loaded.", "green"))
except Exception as e:
    print(colored("Error: OPENROUTER_API_KEY not found in Secrets.", "red"))

# モデル名の定義
MODEL_NAME = "deepseek/deepseek-chat-v3-0324"

# ツール (関数) の定義

LLM が外部ツールとして利用する Python 関数を定義します。LangChain の `@tool` デコレータを使用することで、関数の型ヒントとドキュメント文字列からツールスキーマが生成されます。

ここでは、単純な計算処理を行う `calculate_add` 関数を実装し、エージェントが数値計算リクエストに対してこの関数を呼び出すように設計します。

In [ ]:
from langchain_core.tools import tool

@tool
def calculate_add(a: int, b: int) -> int:
    """
    2つの整数の加算を行います。
    ユーザーが計算を求めた場合に使用されます。

    Args:
        a: 加算する1つ目の整数
        b: 加算する2つ目の整数
    """
    result = a + b
    # 実行確認用ログ
    print(colored(f"[Function Execution] {a} + {b} = {result}", "cyan"))
    return result

# ツールリストの作成
tools = [calculate_add]

# スキーマの確認
print(f"Tool Name: {tools[0].name}")
print(f"Tool Args: {tools[0].args}")

# ツール (関数) の定義

LLM が外部ツールとして利用する Python 関数を定義します。LangChain の `@tool` デコレータを使用することで、関数の型ヒントとドキュメント文字列からツールスキーマが生成されます。

ここでは、単純な計算処理を行う `calculate_add` 関数を実装し、エージェントが数値計算リクエストに対してこの関数を呼び出すように設計します。

In [ ]:
from langchain_core.tools import tool

@tool
def calculate_add(a: int, b: int) -> int:
    """
    2つの整数の加算を行います。
    ユーザーが計算を求めた場合に使用されます。

    Args:
        a: 加算する1つ目の整数
        b: 加算する2つ目の整数
    """
    result = a + b
    # 実行確認用ログ
    print(colored(f"[Function Execution] {a} + {b} = {result}", "cyan"))
    return result

# ツールリストの作成
tools = [calculate_add]

# スキーマの確認
print(f"Tool Name: {tools[0].name}")
print(f"Tool Args: {tools[0].args}")

# LangGraph によるグラフ構築

バックエンドのロジックをステートグラフとして定義します。

### 構成要素
* **State**: 会話履歴 (`messages`) を保持するデータ構造。
* **Chatbot Node**: LLM を呼び出し、応答またはツール呼び出しリクエストを生成するノード。
* **Tools Node**: LLM からのリクエストに基づき、実際の Python 関数を実行するノード。
* **Conditional Edge**: LLM の出力にツール呼び出しが含まれるかを判定し、次に行うべき処理（ツール実行または終了）へ分岐させるロジック。

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition

# State 定義: メッセージ履歴を管理
class State(TypedDict):
    messages: Annotated[list, add_messages]

# LLM の初期化
llm = ChatOpenAI(
    model=MODEL_NAME,
    openai_api_base="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"]
)

# ツールバインディング: LLM に利用可能なツール情報を渡す
llm_with_tools = llm.bind_tools(tools)

# ノード関数: チャットボットの応答生成
def chatbot_node(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# ノード関数: ツールの実行（LangGraph 組み込みの ToolNode を使用）
tool_node = ToolNode(tools)

# グラフビルダーの初期化
graph_builder = StateGraph(State)

# ノードの追加
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_node("tools", tool_node)

# エッジの定義
graph_builder.add_edge(START, "chatbot")

# 条件付きエッジ: ツール呼び出しの有無による分岐
# tools_condition は、メッセージに tool_calls が含まれていれば "tools" へ、
# そうでなければ END へ遷移する判定ロジック
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

# ツール実行後は再び chatbot ノードへ戻り、結果を言語化させる
graph_builder.add_edge("tools", "chatbot")

# グラフのコンパイル
graph = graph_builder.compile()

# コンソールでの動作検証

Gradio 実装の前に、グラフが正常に動作するかをスクリプト上で確認します。
入力に対してツール呼び出しが発生し、その結果が LLM に渡されているかを確認します。

In [ ]:
from langchain_core.messages import HumanMessage

# 検証用クエリ
query = "3と5を足してください。"
print(f"Input: {query}\n" + "-"*30)

inputs = {"messages": [HumanMessage(content=query)]}

# グラフの実行とストリーミング出力
for event in graph.stream(inputs, stream_mode="values"):
    message = event["messages"][-1]
    msg_type = type(message).__name__

    print(f"Step Output [{msg_type}]:")

    # ツール呼び出し情報の表示
    if hasattr(message, "tool_calls") and message.tool_calls:
        for tool_call in message.tool_calls:
            print(colored(f"  Tool Call: {tool_call['name']} args={tool_call['args']}", "yellow"))

    # コンテンツの表示
    if message.content:
        print(f"  Content: {message.content}")

    print("-" * 20)

# Gradio による Web UI の実装

構築した LangGraph アプリケーションを Web ブラウザから利用可能なチャット UI として実装します。
Gradio の `ChatInterface` を使用することで、チャット履歴の表示や入力フォームなどの標準的な UI コンポーネントを容易に構築できます。

### 実装のポイント
* **ハンドラ関数**: Gradio からの入力を受け取り、LangGraph を実行し、結果のテキストのみを抽出して UI に返す関数 (`process_chat`) を定義します。
* **UI 構成**: `gr.Blocks` を使用してレイアウトを定義し、タイトルや説明文を追加します。

In [ ]:
import gradio as gr
from langchain_core.messages import HumanMessage

def process_chat(message, history):
    """
    Gradio の入力ハンドラ。
    type="messages" を使用しない場合、history は [[user_msg, bot_msg], ...] の形式で渡されますが、
    今回は LangGraph に現在のメッセージだけを渡す単純な構成のため、history は使用しません。

    Args:
        message (str): ユーザーの現在の入力
        history (list): 過去の会話履歴
    """
    # グラフへの入力データ作成
    inputs = {"messages": [HumanMessage(content=message)]}

    # グラフの実行
    final_state = graph.invoke(inputs)

    # 最終メッセージの内容を返却
    return final_state["messages"][-1].content

# UI 構成
with gr.Blocks() as demo:
    gr.Markdown("# LangGraph Tool Calling UI")
    gr.Markdown(
        "LangGraph バックエンドによるツール実行デモです。\n"
        "入力例: **「3と5を足して」** と入力すると、内部で Python 関数が実行され、結果が返されます。"
    )

    chat_interface = gr.ChatInterface(
        fn=process_chat,
        # type="messages" を削除 (互換性確保のため)
        examples=["3と5を足してください", "こんにちは"],
    )

if __name__ == "__main__":
    # share=True に設定することで、外部アクセス可能なURL (例: https://xxxxx.gradio.live) が発行されます
    demo.launch(share=True)